In [18]:
from glob import glob
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta
import codecs

def make_river_flux(place_list):
    """
水質水文データベースから各月ごとにdat形式でダウウンロードしたデータをfvcom-toolbox
で処理できる形のcsvに出力する。datファイル名に統一した名前を入れ、その名前をlistで
引数に入れる。
    """
    df = pd.DataFrame()
    for place in place_list:
        flux = []
        paths =glob('../'+place+'*') #datがあるpathを指定してください！
        print(paths)
        #一日のデータをresの各行で格納
        res = []
        for path in paths:
            with codecs.open(path,'r','utf-8','ignore') as f:
                l = f.readlines()
                for i in range(10,len(l)): #11行目以降が時系列データになっています。
                    res.append(l[i].split(','))
        


        #一日のデータのうち、空白や記号を取り除きます。奇数番目にデータが入っている
        data = []
        for i in range(len(res)):
            data_cld = [res[i][j].replace('    ','') for j in range(1,len(res[i]),2)]
            data.append(data_cld)
        
        #一行のlistにまとめる
        flux = [float(data[i][j]) for i in range(len(res))for j in range(24)]

        datelist = []
        for x in range((len(res))*24):
            t = dt.strptime(res[0][0], '%Y/%m/%d')+ timedelta(hours=1) \
                    +timedelta(hours=x) #初期時間からhours=xだけずらした時間
            t = t.strftime('%d-%m-%Y %H:%M') #書式の変換
            datelist.append(t)
            
        #create df

        df['datetime'] = datelist
        print(place)
        df[place] = flux
        #df.to_csv('../river_flux2020.csv')
    return df

flux = make_river_flux(['tamagawa'])
print(flux)

['..\\tamagawa202001.dat', '..\\tamagawa202002.dat', '..\\tamagawa202003.dat', '..\\tamagawa202004.dat', '..\\tamagawa202005.dat', '..\\tamagawa202006.dat', '..\\tamagawa202007.dat', '..\\tamagawa202008.dat', '..\\tamagawa202009.dat', '..\\tamagawa202010.dat', '..\\tamagawa202011.dat', '..\\tamagawa202012.dat']
tamagawa
              datetime  tamagawa
0     01-01-2020 01:00      0.60
1     01-01-2020 02:00      0.55
2     01-01-2020 03:00      0.56
3     01-01-2020 04:00      0.66
4     01-01-2020 05:00      0.86
...                ...       ...
8779  31-12-2020 20:00      1.37
8780  31-12-2020 21:00      0.94
8781  31-12-2020 22:00      0.63
8782  31-12-2020 23:00      0.53
8783  01-01-2021 00:00      0.46

[8784 rows x 2 columns]


In [19]:
flux = make_river_flux(['tamagawa','ishihara'])
flux

['..\\tamagawa202001.dat', '..\\tamagawa202002.dat', '..\\tamagawa202003.dat', '..\\tamagawa202004.dat', '..\\tamagawa202005.dat', '..\\tamagawa202006.dat', '..\\tamagawa202007.dat', '..\\tamagawa202008.dat', '..\\tamagawa202009.dat', '..\\tamagawa202010.dat', '..\\tamagawa202011.dat', '..\\tamagawa202012.dat']
tamagawa
['..\\ishihara202001.dat', '..\\ishihara202002.dat', '..\\ishihara202003.dat', '..\\ishihara202004.dat', '..\\ishihara202005.dat', '..\\ishihara202006.dat', '..\\ishihara202007.dat', '..\\ishihara202008.dat', '..\\ishihara202009.dat', '..\\ishihara202010.dat', '..\\ishihara202011.dat', '..\\ishihara202012.dat']
ishihara


,datetime,tamagawa,ishihara
0,01-01-2020 01:00,0.60,0.57
1,01-01-2020 02:00,0.55,0.56
2,01-01-2020 03:00,0.56,0.56
3,01-01-2020 04:00,0.66,0.55
4,01-01-2020 05:00,0.86,0.55
...,...,...,...
8779,31-12-2020 20:00,1.37,0.63
8780,31-12-2020 21:00,0.94,0.64
8781,31-12-2020 22:00,0.63,0.63
8782,31-12-2020 23:00,0.53,0.63


In [28]:
def level2flux(df):
    columns = df.columns
    if 'ishihara' in columns:
        df['Tamagawa'] = (-21.004*(df['ishihara']**3) +264.5*(df['ishihara']**2) -313.76*df['ishihara']+106.7)/1.76
    #大芦橋と菅間の流量の和を荒川から東京湾への流入量とした
    #Q隅田川 = 0.6荒川
    return df    

Tamagawa = level2flux(flux)
Tamagawa

,datetime,tamagawa,ishihara,Tamagawa
0,01-01-2020 01:00,0.60,0.57,5.626736
1,01-01-2020 02:00,0.55,0.56,5.825546
2,01-01-2020 03:00,0.56,0.56,5.825546
3,01-01-2020 04:00,0.66,0.55,6.050403
4,01-01-2020 05:00,0.86,0.55,6.050403
...,...,...,...,...
8779,31-12-2020 20:00,1.37,0.63,4.976854
8780,31-12-2020 21:00,0.94,0.64,4.958368
8781,31-12-2020 22:00,0.63,0.63,4.976854
8782,31-12-2020 23:00,0.53,0.63,4.976854


In [24]:
#大芦橋と菅間の流量の和を荒川から東京湾への流入量とした